Setup:
Consider the 1-d boundary value problem from fluid dynamics:
$-u''(x)+V(x)u'(x)=f(x),x \in [0,1]$ <br>
$u(0) = u(1) = 1$ <br>
and we will set: <br>
V(x) = $\gamma$ <br>
f(x) = 1

Part 1: Problem formulation: <br>
a: write this as a variational problem: $A(u,\phi)=F(\phi)$
for some test function $\phi$ <br>



b: Take $\phi_i$ to be the hat functions discussed in class and aproximate u as a linear combination of these basis vectors: $u(x)=\sum_i u_i \phi_i(x)$. Show that part (a) becomes a linear algebra problem of the form Ax=b.
Derive expressions for the matrix A and bector b.
Show $A_ji = A(\phi_j,\phi_i)$ and can be writte as the sum of two matrices A = A<sub>1</sub> + A<sub>2</sub> where A<sub>1</sub> and A<sub>2</sub> corresopond to the first two terms on the left hand side of the ODE

c: Implement a routine that will return A as a sparse matrix and b given n and $\gamma$